# introduce

- public_api提供公共Api,无需认证/授权

In [2]:
import ccxt
exchange_ins = ccxt.binance()
exchange_ins

ccxt.binance()

#  time
返回交易所毫秒级时间戳

In [21]:
exchange_ins.fetch_time()

1723444923238

# markets

- In ccxt：
  - every exchange offers multiple markets(coin) within itself.
  - Each market is defined by two or more currencies(USDT,USDC).


## Load all


In [4]:
# 注意 此方法会产生本地缓存,第二次调用使用缓存
market = exchange_ins.load_markets()

In [ ]:
# 无缓存
market = exchange_ins.load_markets(reload=True)

## Get One


In [ ]:
btc_market = exchange_ins.markets["BTC/USDT"]
btc_market

{'id': 'BTCUSDT',
 'lowercaseId': 'btcusdt',
 'symbol': 'BTC/USDT',
 'base': 'BTC',
 'quote': 'USDT',
 'settle': None,
 'baseId': 'BTC',
 'quoteId': 'USDT',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'index': None,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'subType': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 1e-05,
  'price': 0.01,
  'cost': None,
  'base': 1e-08,
  'quote': 1e-08},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 1e-05, 'max': 9000.0},
  'price': {'min': 0.01, 'max': 1000000.0},
  'cost': {'min': 5.0, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 108.87919041}},
 'created': None,
 'info': {'symbol': 'BTCUSDT',
  'status': 'TRADING',
  'baseAsset': 'BTC',
  'baseAssetPrecision': '8',
  'quoteAsset': 'USDT',
  '

In [20]:
exchange_ins.markets.get("ETH/USDT",{}).get("spot")

True

## Advanced Filter


In [ ]:
from typing import Any


def get_markets(
    spot_only: bool = False,
    margin_only: bool = False,
    futures_only: bool = False,
    tradable_only: bool = True,
    active_only: bool = False,
) -> dict[str, Any]:
    """
    Return exchange ccxt markets, filtered out by base currency and quote currency
    if this was requested in parameters.
    """
    markets = exchange_ins.load_markets()
    if not markets:
        raise RuntimeError("Markets were not loaded.")

    if spot_only:
        markets = {k: v for k, v in markets.items() if v.get("spot", False) is True}
    if margin_only:
        markets = {k: v for k, v in markets.items() if v.get("margin", False) is True}
    if futures_only:
        #  the contract is a linear contract (settled in quote currency)
        markets = {k: v for k, v in markets.items() if v.get("linear", False) is True}
    if active_only:
        markets = {
            k: v for k, v in markets.items() if v.get("active", True) is not False
        }

    return markets

In [ ]:
spot = get_markets(spot_only=True)
spot.get("ETH/USDT:USDT")  # 获取合约,返回空

In [ ]:
spot.get("ETH/USDT")  # 获取现货

{'id': 'ETHUSDT',
 'lowercaseId': 'ethusdt',
 'symbol': 'ETH/USDT',
 'base': 'ETH',
 'quote': 'USDT',
 'settle': None,
 'baseId': 'ETH',
 'quoteId': 'USDT',
 'settleId': None,
 'type': 'spot',
 'spot': True,
 'margin': True,
 'swap': False,
 'future': False,
 'option': False,
 'index': None,
 'active': True,
 'contract': False,
 'linear': None,
 'inverse': None,
 'subType': None,
 'taker': 0.001,
 'maker': 0.001,
 'contractSize': None,
 'expiry': None,
 'expiryDatetime': None,
 'strike': None,
 'optionType': None,
 'precision': {'amount': 4, 'price': 2, 'cost': None, 'base': 8, 'quote': 8},
 'limits': {'leverage': {'min': None, 'max': None},
  'amount': {'min': 0.0001, 'max': 9000.0},
  'price': {'min': 0.01, 'max': 1000000.0},
  'cost': {'min': 5.0, 'max': 9000000.0},
  'market': {'min': 0.0, 'max': 2382.977595}},
 'created': None,
 'info': {'symbol': 'ETHUSDT',
  'status': 'TRADING',
  'baseAsset': 'ETH',
  'baseAssetPrecision': '8',
  'quoteAsset': 'USDT',
  'quotePrecision': '8',
 

# ohlcv


In [ ]:
from datetime import datetime, timedelta

# 一天前
time_stamp = int((datetime.now() - timedelta(days=1)).timestamp()) * 1000
print(time_stamp)
ohlcv = exchange_ins.fetch_ohlcv(
    symbol="BTC/USDT:USDT", timeframe="5m", since=time_stamp, limit=300, params={}
)
print(type(ohlcv))  # <class 'list'>

1720690821000
<class 'list'>


In [ ]:
import pandas as pd

DEFAULT_DATAFRAME_COLUMNS = ["date", "open", "high", "low", "close", "volume"]
df = pd.DataFrame(ohlcv, columns=DEFAULT_DATAFRAME_COLUMNS)
df["date"] = pd.to_datetime(df["date"], unit="ms", utc=True)
df

,date,open,high,low,close,volume
0,2024-07-11 09:45:00+00:00,58207.4,58212.8,58091.1,58173.6,468.517
1,2024-07-11 09:50:00+00:00,58173.7,58222.6,58139.3,58213.7,282.827
2,2024-07-11 09:55:00+00:00,58213.7,58293.5,58205.6,58265.7,468.394
3,2024-07-11 10:00:00+00:00,58265.8,58489.8,58239.6,58366.2,2218.911
4,2024-07-11 10:05:00+00:00,58366.2,58376.9,58213.0,58260.0,1013.294
...,...,...,...,...,...,...
283,2024-07-12 09:20:00+00:00,57153.6,57196.4,57145.5,57166.5,157.885
284,2024-07-12 09:25:00+00:00,57166.6,57184.8,57142.8,57161.6,166.331
285,2024-07-12 09:30:00+00:00,57161.9,57188.6,57081.7,57150.1,430.339
286,2024-07-12 09:35:00+00:00,57150.0,57157.1,57000.0,57087.0,1510.345


# order book

In [9]:
exchange_ins.fetch_order_book("ZRO/USDT",limit=10)

{'symbol': 'ZRO/USDT',
 'bids': [[3.272, 1280.97],
  [3.271, 1598.43],
  [3.27, 1985.1],
  [3.269, 1858.48],
  [3.268, 2344.07],
  [3.267, 3882.77],
  [3.266, 3768.7],
  [3.265, 813.52],
  [3.264, 1672.9],
  [3.263, 2598.61]],
 'asks': [[3.273, 12.0],
  [3.274, 512.28],
  [3.275, 2501.72],
  [3.276, 2417.64],
  [3.277, 958.78],
  [3.278, 2755.09],
  [3.279, 861.09],
  [3.28, 2563.11],
  [3.281, 852.8],
  [3.282, 1568.27]],
 'timestamp': None,
 'datetime': None,
 'nonce': 316242461}

In [10]:
# L2 order book的特点: 相同价格的订单会被合并到一起
exchange_ins.fetch_l2_order_book("ZRO/USDT", limit=10)

{'symbol': 'ZRO/USDT',
 'bids': [[3.272, 1280.97],
  [3.271, 1598.43],
  [3.27, 2190.41],
  [3.269, 1958.48],
  [3.268, 2344.07],
  [3.267, 3882.77],
  [3.266, 3768.7],
  [3.265, 813.52],
  [3.264, 1672.9],
  [3.263, 2598.61]],
 'asks': [[3.273, 12.0],
  [3.274, 512.28],
  [3.275, 2501.72],
  [3.276, 2417.64],
  [3.277, 958.78],
  [3.278, 2755.09],
  [3.279, 861.09],
  [3.28, 2563.11],
  [3.281, 852.8],
  [3.282, 1568.27]],
 'timestamp': None,
 'datetime': None,
 'nonce': 316242472}

# ticker

过去24小时行情数据


In [26]:
btc_ticker = exchange_ins.fetch_ticker("ZRO/USDT")
btc_ticker

{'symbol': 'ZRO/USDT',
 'timestamp': 1723627679631,
 'datetime': '2024-08-14T09:27:59.631Z',
 'high': 3.624,
 'low': 3.374,
 'bid': 3.54,
 'bidVolume': 309.28,
 'ask': 3.541,
 'askVolume': 578.15,
 'vwap': 3.51122803,
 'open': 3.387,
 'close': 3.54,
 'last': 3.54,
 'previousClose': 3.387,
 'change': 0.153,
 'percentage': 4.517,
 'average': 3.4635,
 'baseVolume': 4369826.77,
 'quoteVolume': 15343458.24846,
 'info': {'symbol': 'ZROUSDT',
  'priceChange': '0.15300000',
  'priceChangePercent': '4.517',
  'weightedAvgPrice': '3.51122803',
  'prevClosePrice': '3.38700000',
  'lastPrice': '3.54000000',
  'lastQty': '255.29000000',
  'bidPrice': '3.54000000',
  'bidQty': '309.28000000',
  'askPrice': '3.54100000',
  'askQty': '578.15000000',
  'openPrice': '3.38700000',
  'highPrice': '3.62400000',
  'lowPrice': '3.37400000',
  'volume': '4369826.77000000',
  'quoteVolume': '15343458.24846000',
  'openTime': '1723541279631',
  'closeTime': '1723627679631',
  'firstId': '11839545',
  'lastId': 

In [17]:
btc_ticker = exchange_ins.fetch_ticker("ZRO/USDT:USDT")
btc_ticker

{'symbol': 'ZRO/USDT:USDT',
 'timestamp': 1723625541570,
 'datetime': '2024-08-14T08:52:21.570Z',
 'high': 3.62,
 'low': 3.37,
 'bid': None,
 'bidVolume': None,
 'ask': None,
 'askVolume': None,
 'vwap': 3.5004767,
 'open': 3.397,
 'close': 3.531,
 'last': 3.531,
 'previousClose': None,
 'change': 0.134,
 'percentage': 3.945,
 'average': 3.464,
 'baseVolume': 18165752.4,
 'quoteVolume': 63588792.7041936,
 'info': {'symbol': 'ZROUSDT',
  'priceChange': '0.1340000',
  'priceChangePercent': '3.945',
  'weightedAvgPrice': '3.5004767',
  'lastPrice': '3.5310000',
  'lastQty': '169.7',
  'openPrice': '3.3970000',
  'highPrice': '3.6200000',
  'lowPrice': '3.3700000',
  'volume': '18165752.4',
  'quoteVolume': '63588792.7041936',
  'openTime': '1723539120000',
  'closeTime': '1723625541570',
  'firstId': '56042043',
  'lastId': '56305458',
  'count': '263416'}}

# public trades

返回一个有序的交易列表（一个平面数组，按时间戳升序排序，最旧的交易在前，最近的交易在最后）

In [20]:
exchange_ins.fetch_trades("ZRO/USDT",limit=1)

[{'info': {'a': '6685169',
   'p': '3.23300000',
   'q': '68.85000000',
   'f': '11699129',
   'l': '11699134',
   'T': '1723444840011',
   'm': True,
   'M': True},
  'timestamp': 1723444840011,
  'datetime': '2024-08-12T06:40:40.011Z',
  'symbol': 'ZRO/USDT',
  'id': '6685169',
  'order': None,
  'type': None,
  'side': 'sell',
  'takerOrMaker': None,
  'price': 3.233,
  'amount': 68.85,
  'cost': 222.59205,
  'fee': None,
  'fees': []}]

# open Interest

当前合约持仓量

In [27]:
exchange_ins.fetch_open_interest("ZRO/USDT:USDT")

{'symbol': 'ZRO/USDT:USDT',
 'baseVolume': 6056009.4,
 'quoteVolume': None,
 'openInterestAmount': 6056009.4,
 'openInterestValue': None,
 'timestamp': 1723628205330,
 'datetime': '2024-08-14T09:36:45.330Z',
 'info': {'symbol': 'ZROUSDT',
  'openInterest': '6056009.4',
  'time': '1723628205330'}}

In [4]:
exchange_ins.fetch_open_interest_history("AEVO/USDT:USDT",timeframe="1d", limit=2)

[{'symbol': 'AEVO/USDT:USDT',
  'baseVolume': 30155876.5,
  'quoteVolume': 10132856.998024,
  'openInterestAmount': 30155876.5,
  'openInterestValue': 10132856.998024,
  'timestamp': 1723420800000,
  'datetime': '2024-08-12T00:00:00.000Z',
  'info': {'symbol': 'AEVOUSDT',
   'sumOpenInterest': '30155876.50000000',
   'sumOpenInterestValue': '10132856.99802400',
   'timestamp': '1723420800000'}},
 {'symbol': 'AEVO/USDT:USDT',
  'baseVolume': 32062720.0,
  'quoteVolume': 11956678.847616,
  'openInterestAmount': 32062720.0,
  'openInterestValue': 11956678.847616,
  'timestamp': 1723507200000,
  'datetime': '2024-08-13T00:00:00.000Z',
  'info': {'symbol': 'AEVOUSDT',
   'sumOpenInterest': '32062720.00000000',
   'sumOpenInterestValue': '11956678.84761600',
   'timestamp': '1723507200000'}}]